# Nearly incompressible elasticity

We investigate how nearly incompressible materials (the limit $\lambda\to\infty$, or equivalently $\nu\to 1/2$) yields to so-called Poisson (or volume) locking for standard elasticity.

We cure the problem by using the Stokes-stable pair $Q^1-P^0$ on quadrilaterals, which is equivalently to reduced integration of the problematic divergence term.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import MakeStructured2DMesh

A rectangular mesh consisting of quadrilaterals:

In [ ]:
mapping = lambda x,y : (x,0.3*y)
mesh = MakeStructured2DMesh(quads=True, nx=10, ny=2, mapping=mapping)
Draw(mesh);

In [ ]:
# Young's modulus and Poisson's ratio
E  = 10
nu = Parameter(0.3)

We define the solving procedure, where the polynomial order can be specified as well as if we want to do a projection of the divergence term of linear elasticity.

In [ ]:
def Solve(mesh, order=1, projection=False):
    fes = VectorH1(mesh, order=order, dirichlet="left")
    u,v = fes.TnT()
    
    mu  = E / 2 / (1+nu)
    lam = E * nu / ((1+nu)*(1-2*nu))

    a = BilinearForm(fes, symmetric=True)
    a += 2*mu*InnerProduct(Sym(Grad(u)),Sym(Grad(v)))*dx
    if projection:
        #a += lam*Interpolate(div(u),L2(mesh,order=0))*Interpolate(div(v),L2(mesh,order=0))*dx
        # alternatively: use reduced integration (equivalent to L2-projection)
        a += lam*div(u)*div(v)*dx(bonus_intorder=-1)
    else:
        a += lam*div(u)*div(v)*dx

    f = LinearForm(fes)
    f += 0.15*CF((0,x))*v*dx

    gfu = GridFunction(fes)
    a.Assemble()
    f.Assemble()

    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    return gfu

Setting $\nu$ close to $0.5$ will lead to so-called Poisson (volume) locking. When using the projection the locking issue is reduced. Also increasing the polynomial degree helps.

In [ ]:
nu.Set(0.4)
gfu = Solve(mesh, order=1, projection=False)
Draw(gfu, mesh, deformation=True);

### Explanation
We can define the pressure $p=\lambda\mathrm{div} u$ as an additional unknown, which translates the elasticity problem into a mixed Stokes saddle point problem with penalty: Find $(u,p)$ such that for all $(v,q)$
\begin{align*}
&\int_{\Omega} 2\mu\,\varepsilon(u):\varepsilon(v)\,dx &&+\int_{\Omega} p\,\mathrm{div} v\,dx &&= \int_{\Omega}f\cdot v\,dx\\
&\int_{\Omega} q\,\mathrm{div} u\,dx &&-\int_{\Omega}\frac{1}{\lambda}pq\,dx &&= 0.
\end{align*}
The limit $\lambda\to\infty$ is now well-defined, which is the Stokes problem.

We can use the Stokes stable discretization pair $(u_h,p_h)\in Q^1\times P^0$, where $Q^1$ is the space of piecewise bi-linear polynomials and $P^0$ the space of piecewise constants, to solve the saddle point problem. However, looking at the second equation, we observe that it is equivalent to the $L^2$-projection of $\mathrm{div} u$ onto piecewise constants, denoted by $\Pi^0$, i.e.
\begin{align*}
p = \lambda\,\Pi^0\mathrm{div} u.
\end{align*}
Reinserting this into the original elasticity problem we obtain
\begin{align*}
&\int_{\Omega} 2\mu\,\varepsilon(u):\varepsilon(v)+ \lambda\, \Pi^0(\mathrm{div} u)\,\Pi^0(\mathrm{div} v)\,dx = \int_{\Omega}f\cdot v\,dx,
\end{align*}
which is robust in the nearly incompressible case. Note that the $L^2$-projection corresponds to reduced integration, using less Gauss integration points.

This concept of under-integrating a penalty term to weaken it, is a common idea to overcome locking issues. However, one needs to be careful to not induce so-called spurious zero energy modes, which would destroy stability because coercivity would be lost (note, this cannot happen here, as the first term $\varepsilon(u)$ controls the divergence).